In [3]:
import torch
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from alive_progress import alive_bar

In [4]:
class IrisDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        x = self.features[index]
        y = self.labels[index]
        return x, y

In [5]:
class IrisClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
# Load the Iris dataset
iris = load_iris()
features = iris.data
labels = iris.target

# Split the dataset into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

# Convert the data to PyTorch tensors
train_features = torch.tensor(train_features, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)
test_features = torch.tensor(test_features, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.long)

# Create the datasets
train_dataset = IrisDataset(train_features, train_labels)
test_dataset = IrisDataset(test_features, test_labels)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [7]:
# Example usage
for features, labels in train_loader:
    print("Features:", features.shape)
    print("Labels:", labels.shape)
    break

Features: torch.Size([32, 4])
Labels: torch.Size([32])


In [8]:

input_size = 4
hidden_size = 10
num_classes = 3
model = IrisClassifier(input_size, hidden_size, num_classes)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for features, labels in train_loader:
        # Forward pass
        outputs = model(features)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Testing loop
model.eval()

Epoch [10/100], Loss: 0.3883
Epoch [20/100], Loss: 0.1852
Epoch [30/100], Loss: 0.1242
Epoch [40/100], Loss: 0.0849
Epoch [50/100], Loss: 0.0478
Epoch [60/100], Loss: 0.0740
Epoch [70/100], Loss: 0.0531
Epoch [80/100], Loss: 0.1336
Epoch [90/100], Loss: 0.0499
Epoch [100/100], Loss: 0.0361


IrisClassifier(
  (fc1): Linear(in_features=4, out_features=10, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)

In [9]:
def convert_to_onnx(model, input_size, file_name="iris_model.onnx"):
    # Import required libraries
    import torch
    import torch.onnx

    # Create a dummy input tensor
    dummy_input = torch.randn(1, input_size)

    # Export the model to ONNX format
    torch.onnx.export(model,               # model being run
                      dummy_input,         # model input (or a tuple for multiple inputs)
                      file_name,           # where to save the model
                      export_params=True,  # store the trained parameter weights inside the model file
                      opset_version=10,    # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names=['input'],   # the model's input names
                      output_names=['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})
    
    print(f"Model has been converted to ONNX and saved as {file_name}")

# Example usage:
# convert_to_onnx(model, input_size=4)

In [11]:
convert_to_onnx(model, input_size=4)

Model has been converted to ONNX and saved as iris_model.onnx


In [ ]:
# %%
def verify_onnx_model(pytorch_model, onnx_file, input_size):
    import onnx
    import onnxruntime
    import numpy as np

    # Load the ONNX model
    onnx_model = onnx.load(onnx_file)
    onnx.checker.check_model(onnx_model)

    # Create an ONNX Runtime session
    ort_session = onnxruntime.InferenceSession(onnx_file)

    # Generate a random input
    input_data = np.random.randn(1, input_size).astype(np.float32)

    # Run inference with ONNX Runtime
    ort_inputs = {ort_session.get_inputs()[0].name: input_data}
    ort_outputs = ort_session.run(None, ort_inputs)

    # Run inference with PyTorch
    pytorch_input = torch.tensor(input_data)
    pytorch_model.eval()
    with torch.no_grad():
        pytorch_output = pytorch_model(pytorch_input)

    # Compare the results
    np.testing.assert_allclose(ort_outputs[0], pytorch_output.numpy(), rtol=1e-03, atol=1e-05)
    print("ONNX model verified successfully!")

In [ ]:
# Verify the ONNX model
verify_onnx_model(model, "iris_model.onnx", input_size=4)